In [50]:
from pathlib import Path
import pandas as pd

In [51]:
ghgi_v2_dir = Path("C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code")
data_dir = Path("C:/Users/nkruskamp/Research Triangle Institute/EPA Gridded Methane - Task 2/data")

In [52]:
file_list = []
for x in list(ghgi_v2_dir.glob("*")):
    # there is something wrong with this folder, so skip it. Hence why this is a more complicated loop.
    if "Final_Gridded_Data" in str(x):
        continue
    if x.is_dir():
        file_list.extend(list(x.rglob("*ProxyMapping*.xlsx")))
file_list

[WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code/GEPA_2022_Supplement/GEPA_PostMeter/InputData/PostMeter_ProxyMapping.xlsx'),
 WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code/GEPA_Abandoned_Coal/InputData/AbandonedCoal_ProxyMapping.xlsx'),
 WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code/GEPA_Coal/InputData/Coal_ProxyMapping.xlsx'),
 WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code/GEPA_Combustion_Mobile/InputData/MobileCombustion_ProxyMapping.xlsx'),
 WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_Source_Code/GEPA_Combustion_Stationary/InputData/StationaryCombustion_ProxyMapping.xlsx'),
 WindowsPath('C:/Users/nkruskamp/Environmental Protection Agency (EPA)/Gridded CH4 Inventory - GEPA_So

In [53]:
proxy_df_list = []
for xl_file_path in file_list:
    # get the sector name for the file
    sector_name = xl_file_path.parents[1].name.split("_")[1]
    # get the excel file
    xl = pd.ExcelFile(xl_file_path)
    # get the sheet name if it has proxy mappings
    sheet_name = [x for x in xl.sheet_names if "Proxy Map" in x]
    # if it does have a proxy mapping sheet:
    if sheet_name:
        # read the file, assign the sector name to column. Add to list
        proxy_df_list.append(xl.parse(sheet_name[0], header=1).assign(sector=sector_name))
    else:
        print("sheet not found")

In [54]:
# concat all proxy mapping dataframes together.
proxy_df = pd.concat(proxy_df_list).assign(
    # consolidate different proxy description columns into a single one called description
    description=lambda df: df.loc[:, df.columns.str.contains("Proxy_Description")]
    .astype(str)
    .agg(", ".join, axis=1),
)
# remove old proxy description columns
proxy_df = proxy_df.loc[:, ~df.columns.str.contains("Proxy_Description")]
# save to a file
proxy_df.to_csv(data_dir / "all_proxy_mappings.csv")


In [55]:
ghgi_df_list = []
for xl_file_path in file_list:
    # get the sector name for the file
    sector_name = xl_file_path.parents[1].name.split("_")[1]
    # get the excel file
    xl = pd.ExcelFile(xl_file_path)
    # get the sheet name if it has proxy mappings
    sheet_name = [x for x in xl.sheet_names if "GHGI Map" in x]
    # if it does have a proxy mapping sheet:
    if sheet_name:
        # read the file, assign the sector name to column. Add to list
        ghgi_df_list.append(xl.parse(sheet_name[0], header=1).assign(sector=sector_name))
    else:
        print("sheet not found")

c:\Users\nkruskamp\AppData\Local\mambaforge\envs\disease_raster_align\lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [59]:
ghgi_df = pd.concat(ghgi_df_list).assign(
    # consolidate different proxy description columns into a single one called description
    description=lambda df: df.loc[:, df.columns.str.contains("Proxy_Description")]
    .astype(str)
    .agg(", ".join, axis=1),
)
ghgi_df.to_csv(data_dir / "all_ghgi_mappings.csv")